In [1]:
!pip install google-cloud-vision
!pip install requests
!pip install openai
!pip install --upgrade google-cloud-vision google-cloud-translate

  Obtaining dependency information for google-cloud-vision from https://files.pythonhosted.org/packages/04/fc/1e2dfc127d6178b2aed6066bd5b556f4c9dd126d027e0d9855d8078cdeef/google_cloud_vision-3.7.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.1 from https://files.pythonhosted.org/packages/2d/ed/e514e0c59cdf1a469b1a1ab21b77698d0692adaa7cbc920c3a0b287e8493/google_api_core-2.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth!=2.24.0,!=2.25.0,<3.0.0dev,>=2.14.1 from https://files.pythonhosted.org/packages/9e/8d/ddbcf81ec751d8ee5fd18ac11ff38a0e110f39dfbf105e6d9db69d556dd0/google_auth-2.29.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0dev,>=1.22.3 from https://files.pythonhosted.org/packages/ad/41/7361075f3a31dcd05a6a38cfd807a6eecbfb6dbfe420d922cd400fc03ac1/proto_plus-1.23.

In [4]:
from google.cloud import vision
from google.oauth2 import service_account
import io
import requests
import pandas as pd

In [26]:
df=pd.read_csv('큐레이션통합6.csv')
df

,img,curHead,curArtist,curTitle,curColor,curPer,curExplain,curObject
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[13, 13, 13]",56.892578,NaN,NaN
1,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[116, 116, 116]",36.462674,NaN,NaN
2,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[193, 193, 193]",6.644748,NaN,NaN
3,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,박재범,Like I Do,"[13, 13, 13]",56.892578,NaN,NaN
4,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,박재범,Like I Do,"[116, 116, 116]",36.462674,NaN,NaN
...,...,...,...,...,...,...,...,...
64480,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,NeonDreams(네온드림즈),High School Dropout,NaN,NaN,#신나는 #플레이리스트 #여름,NaN
64481,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,NeonDreams(네온드림즈),High School Dropout,NaN,NaN,#신나는 #플레이리스트 #여름,NaN
64482,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Zerb,With You,NaN,NaN,#신나는 #플레이리스트 #여름,NaN
64483,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Zerb,With You,NaN,NaN,#신나는 #플레이리스트 #여름,NaN


In [12]:
len(df['img'].unique())

1110

In [25]:
import pandas as pd
import requests
from google.cloud import vision
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate
import tqdm
# 서비스 계정 키 파일 경로 설정
credentials = service_account.Credentials.from_service_account_file('impactful-bee-425206-q9-f096b326d842.json')

# Vision API 클라이언트 생성
vision_client = vision.ImageAnnotatorClient(credentials=credentials)

# Translate API 클라이언트 생성
translate_client = translate.Client(credentials=credentials)



# 이미지 URL의 중복 제거
unique_img_urls = df['img'].unique().tolist()

# 이미지 정보 처리 함수
def process_image(img_url):
    # 이미지 URL로부터 이미지 로드
    response = requests.get(img_url)
    content = response.content

    # 이미지 객체 생성
    image = vision.Image(content=content)

    # 라벨 감지
    label_response = vision_client.label_detection(image=image)
    labels = label_response.label_annotations

    # 라벨 설명을 한국어로 번역
    descriptions = [label.description for label in labels]
    if descriptions:
        translations = translate_client.translate(descriptions, target_language='ko')
        translated_descriptions = [translation['translatedText'] for translation in translations]
    else:
        translated_descriptions = []

    # 색감 정보 요청
    color_response = vision_client.image_properties(image=image)
    props = color_response.image_properties_annotation

    # 상위 3개 색감 선택 및 정보 추출
    colors = props.dominant_colors.colors
    sorted_colors = sorted(colors, key=lambda color: color.pixel_fraction, reverse=True)[:3]
    
    color_info = [(color.color.red, color.color.green, color.color.blue) for color in sorted_colors]
    color_ratios = [color.pixel_fraction * 100 for color in sorted_colors]

    return [img_url, translated_descriptions, color_info, color_ratios]
    
# 각 이미지 URL에 대해 process_image 함수 적용
results = [process_image(url) for url in unique_img_urls]

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results, columns=['img', '이미지_객체', '이미지_색감', '색감_비율'])

# '이미지_색감' 및 '색감_비율' 컬럼을 분해하여 별도의 행으로 변환
result_df = result_df.explode('이미지_색감').explode('색감_비율')

# '색감_비율' 컬럼에 NaN 값이 들어갈 수 있으므로 이를 필터링
result_df = result_df[result_df['색감_비율'].notna()]

# 결과 확인
result_df

KeyboardInterrupt: 

In [30]:
color=pd.DataFrame()
color['이미지색감']=result_df['이미지_색감'][::3]
color

,이미지색감
0,"(12.0, 12.0, 12.0)"
0,"(123.0, 123.0, 123.0)"
0,"(86.0, 86.0, 86.0)"
1,"(24.0, 24.0, 24.0)"
1,"(50.0, 50.0, 51.0)"
...,...
1108,"(172.0, 166.0, 165.0)"
1108,"(49.0, 48.0, 43.0)"
1109,"(30.0, 24.0, 22.0)"
1109,"(102.0, 130.0, 162.0)"


In [31]:
filtered_df=pd.DataFrame()
for i in range(0, len(result_df), 9):
    filtered_df=pd.concat([filtered_df, result_df.iloc[i:i+3]])
filtered_df 

,img,이미지_객체,이미지_색감,색감_비율
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",50.380951
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",22.214286
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",9.420635
1,https://img.youtube.com/vi/Wym_B7-zAJU/maxresd...,"[플래시 사진, 속눈썹, 몸짓, 마이크로폰, 스타일, 검정색과 흰색, 음악, 오디오...","(24.0, 24.0, 24.0)",39.460316
1,https://img.youtube.com/vi/Wym_B7-zAJU/maxresd...,"[플래시 사진, 속눈썹, 몸짓, 마이크로폰, 스타일, 검정색과 흰색, 음악, 오디오...","(24.0, 24.0, 24.0)",12.65873
...,...,...,...,...
1108,https://img.youtube.com/vi/LiXcXDajCH4/maxresd...,"[소매, 몸짓, 폰트, 하늘, 행복하다, 풍경, 상표, 좌석, 간단한 기계 장치, ...","(201.0, 194.0, 193.0)",31.476191
1108,https://img.youtube.com/vi/LiXcXDajCH4/maxresd...,"[소매, 몸짓, 폰트, 하늘, 행복하다, 풍경, 상표, 좌석, 간단한 기계 장치, ...","(201.0, 194.0, 193.0)",5.301587
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(30.0, 24.0, 22.0)",22.912699
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(30.0, 24.0, 22.0)",5.801587


In [33]:
filtered_df['이미지_색감']=color['이미지색감']
filtered_df.to_csv('image_unique_color_per_obj')

In [27]:
import pandas as pd
import requests
from google.cloud import vision
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate
from tqdm import tqdm

# 서비스 계정 키 파일 경로 설정
credentials = service_account.Credentials.from_service_account_file('impactful-bee-425206-q9-f096b326d842.json')

# Vision API 클라이언트 생성
vision_client = vision.ImageAnnotatorClient(credentials=credentials)

# Translate API 클라이언트 생성
translate_client = translate.Client(credentials=credentials)

# 이미지 URL의 중복 제거
unique_img_urls = df['img'].unique().tolist()

# 이미지 정보 처리 함수
def process_image(img_url):
    def get_image_content(url):
        try:
            response = requests.get(url, timeout=10)  # 10초 타임아웃 설정
            response.raise_for_status()  # 상태 코드가 400이상이면 예외 발생
            return response.content
        except requests.RequestException as e:
            return None

    # 첫 번째 시도
    content = get_image_content(img_url)

    # 첫 번째 시도가 실패한 경우 URL 변경 후 재시도
    if content is None:
        print(f"첫 번째 시도 실패: {img_url}")
        modified_url = img_url.replace('maxresdefault', '0')
        content = get_image_content(modified_url)
        if content is None:
            print(f"두 번째 시도 실패: {modified_url}")
            return [img_url, [], [], []]  # 두 번째 시도도 실패한 경우 빈 리스트 반환

    # 이미지 객체 생성
    image = vision.Image(content=content)

    # 라벨 감지
    label_response = vision_client.label_detection(image=image)
    labels = label_response.label_annotations

    # 라벨 설명을 한국어로 번역
    descriptions = [label.description for label in labels]
    if descriptions:
        translations = translate_client.translate(descriptions, target_language='ko')
        translated_descriptions = [translation['translatedText'] for translation in translations]
    else:
        translated_descriptions = []

    # 색감 정보 요청
    color_response = vision_client.image_properties(image=image)
    props = color_response.image_properties_annotation

    # 상위 3개 색감 선택 및 정보 추출
    colors = props.dominant_colors.colors
    sorted_colors = sorted(colors, key=lambda color: color.pixel_fraction, reverse=True)[:3]
    
    color_info = [(color.color.red, color.color.green, color.color.blue) for color in sorted_colors]
    color_ratios = [color.pixel_fraction * 100 for color in sorted_colors]

    return [img_url, translated_descriptions, color_info, color_ratios]

# 각 이미지 URL에 대해 process_image 함수 적용 (tqdm으로 진행 상황 표시)
results = [process_image(url) for url in tqdm(unique_img_urls, desc="Processing images")]

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results, columns=['img', '이미지_객체', '이미지_색감', '색감_비율'])

# '이미지_색감' 및 '색감_비율' 컬럼을 분해하여 별도의 행으로 변환
result_df = result_df.explode('이미지_색감').explode('색감_비율')

# '색감_비율' 컬럼에 NaN 값이 들어갈 수 있으므로 이를 필터링
result_df = result_df[result_df['색감_비율'].notna()]

# 결과 확인
result_df


Processing images:   5%|███▎                                                         | 61/1110 [01:03<18:35,  1.06s/it]

첫 번째 시도 실패: https://img.youtube.com/vi/mG0iydK-lkU/maxresdefault.jpg



Processing images:   6%|███▊                                                         | 69/1110 [01:11<18:25,  1.06s/it]

첫 번째 시도 실패: https://img.youtube.com/vi/tIt2oSRnbXg/maxresdefault.jpg



Processing images:   7%|████                                                         | 75/1110 [01:17<17:36,  1.02s/it]

...

첫 번째 시도 실패: https://img.youtube.com/vi/IvBtNJdCPNQ/maxresdefault.jpg



Processing images:  72%|███████████████████████████████████████████▏                | 800/1110 [13:47<05:29,  1.06s/it]

첫 번째 시도 실패: https://img.youtube.com/vi/1QEUJ5d7WO8/maxresdefault.jpg



Processing images: 100%|███████████████████████████████████████████████████████████| 1110/1110 [19:11<00:00,  1.04s/it]


,img,이미지_객체,이미지_색감,색감_비율
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",50.380951
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",22.214286
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(12.0, 12.0, 12.0)",9.420635
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(123.0, 123.0, 123.0)",50.380951
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,"[바지, 시력 관리, 고글, 색안경, 검정색과 흰색, 플래시 사진, 스타일, 안경,...","(123.0, 123.0, 123.0)",22.214286
...,...,...,...,...
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(102.0, 130.0, 162.0)",5.801587
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(102.0, 130.0, 162.0)",5.468254
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(84.0, 125.0, 174.0)",22.912699
1109,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,"[하늘, 물, 낮, 식물, 세계, 빛, 자연, 나무, 황혼, 아레칼레스]","(84.0, 125.0, 174.0)",5.801587


In [29]:
result_df.to_csv('img_unique.csv',index=False)

In [49]:
filtered_df.to_csv('이미지_색감_비율.csv',index=False)

In [48]:
df.to_csv('https붙인_img.csv',index=False)

In [47]:
# filtered_df에서 각 img 값에 해당하는 이미지_객체 값들을 리스트로 변환
grouped_objects = filtered_df.groupby('img')['이미지_객체'].apply(list).reset_index()

# df와 병합하여 curObject 컬럼 추가
final_df = pd.merge(df, grouped_objects, on='img', how='left')

# '이미지_객체' 열 이름을 'curObject'로 변경
final_df = final_df.rename(columns={'이미지_객체': 'curObject'})

# 결과 확인
final_df


,img,curHead,curArtist,curTitle,curColor,curPer,curExplain,curObject,curColor
0,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[13, 13, 13]",56.892578,NaN,NaN,"[(12.0, 12.0, 12.0), (123.0, 123.0, 123.0), (8..."
1,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[116, 116, 116]",36.462674,NaN,NaN,"[(12.0, 12.0, 12.0), (123.0, 123.0, 123.0), (8..."
2,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,BIGNaughty,Vancouver,"[193, 193, 193]",6.644748,NaN,NaN,"[(12.0, 12.0, 12.0), (123.0, 123.0, 123.0), (8..."
3,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,박재범,Like I Do,"[13, 13, 13]",56.892578,NaN,NaN,"[(12.0, 12.0, 12.0), (123.0, 123.0, 123.0), (8..."
4,https://img.youtube.com/vi/G8MoUQHat_c/maxresd...,드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,박재범,Like I Do,"[116, 116, 116]",36.462674,NaN,NaN,"[(12.0, 12.0, 12.0), (123.0, 123.0, 123.0), (8..."
...,...,...,...,...,...,...,...,...,...
64480,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,NeonDreams(네온드림즈),High School Dropout,NaN,NaN,#신나는 #플레이리스트 #여름,NaN,"[(30.0, 24.0, 22.0), (102.0, 130.0, 162.0), (8..."
64481,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,NeonDreams(네온드림즈),High School Dropout,NaN,NaN,#신나는 #플레이리스트 #여름,NaN,"[(30.0, 24.0, 22.0), (102.0, 130.0, 162.0), (8..."
64482,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Zerb,With You,NaN,NaN,#신나는 #플레이리스트 #여름,NaN,"[(30.0, 24.0, 22.0), (102.0, 130.0, 162.0), (8..."
64483,https://img.youtube.com/vi/jwPH69NoBP8/maxresd...,영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Zerb,With You,NaN,NaN,#신나는 #플레이리스트 #여름,NaN,"[(30.0, 24.0, 22.0), (102.0, 130.0, 162.0), (8..."
